In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader
from ModelFunctions import train, evaluate, inference

In [2]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp-airi'))

from spinconv import spinconv

In [3]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    keys = ['pos', 'atomic_numbers', 'cell', 'natoms']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

In [4]:
#config
batch_size = 90
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 20

In [5]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [6]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [7]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/all/train/data.lmdb")
training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)

training_generator = DataListLoader(training_set, batch_size=batch_size)

In [8]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data_mod2.lmdbz")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataListLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [9]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

total entries: 460328
info for item: 0
edge_index:............... [2, 2964]
pos:......................   [86, 3]
cell:..................... [1, 3, 3]
atomic_numbers:...........      [86]
natoms:...................        86
cell_offsets:............. [2964, 3]
force:....................   [86, 3]
distances:................    [2964]
fixed:....................      [86]
sid:......................   2472718
tags:.....................      [86]
y_init:...................    6.2825
y_relaxed:................   -0.0256
pos_relaxed:..............   [86, 3]


In [10]:
#model
model = spinconv(None, None, 1, otf_graph=True, regress_forces=False)
#model = OCPDataParallel(model, output_device=0, num_gpus=2)
model = DataParallel(model)
#model = DistributedDataParallel(model, device_ids=[0, 1])
#model = torch.load("id_all_2021-09-24-17-19-54.pickle")

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

In [11]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)    

2021-09-25-11-23-25


In [12]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [13]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

no graph
CPU times: user 177 µs, sys: 71 µs, total: 248 µs
Wall time: 243 µs


## Training

In [ ]:
%%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')

with open('SpinConv_identical.txt', 'a') as f:
    f.write(f'{timestamp}\nStart training model {str(model)}\n' )

for i in range(1, epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))
    with open('SpinConv_identical.txt', 'a') as f:
        f.write(f'{loss[-1]}\n{loss_eval[-1]}\n')

2021-09-25-11-23-25
Start training model DataParallel(
  (module): spinconv(
    (act): Swish()
    (distance_expansion_forces): GaussianSmearing()
    (embeddingblock2): EmbeddingBlock(
      (act): Swish()
      (fc1): Linear(in_features=200, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1600, bias=True)
      (fc3): Linear(in_features=200, out_features=32, bias=True)
      (source_embedding): Embedding(90, 32)
      (target_embedding): Embedding(90, 32)
      (embed_fc1): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=1)
    )
    (distfc1): Linear(in_features=200, out_features=200, bias=True)
    (distfc2): Linear(in_features=200, out_features=200, bias=True)
    (dist_block): DistanceBlock(
      (distance_expansion): GaussianSmearing()
      (dist_scalar): Embedding(8100, 1)
      (dist_offset): Embedding(8100, 1)
      (fc1): Linear(in_features=200, out_features=200, bias=True)
    )
    (message_blocks): ModuleLis

ml-test-server-0:55883:55903 [1] NCCL INFO Channel 11 : 1[bc000] -> 0[57000] via P2P/direct pointer
ml-test-server-0:55883:55902 [0] NCCL INFO Channel 11 : 0[57000] -> 1[bc000] via P2P/direct pointer
ml-test-server-0:55883:55902 [0] NCCL INFO 12 coll channels, 16 p2p channels, 16 p2p channels per peer
ml-test-server-0:55883:55903 [1] NCCL INFO 12 coll channels, 16 p2p channels, 16 p2p channels per peer
ml-test-server-0:55883:55902 [0] NCCL INFO comm 0x7fc09c001060 rank 0 nranks 2 cudaDev 0 busId 57000 - Init COMPLETE
ml-test-server-0:55883:55903 [1] NCCL INFO comm 0x7fc098001060 rank 1 nranks 2 cudaDev 1 busId bc000 - Init COMPLETE
ml-test-server-0:55883:55883 [0] NCCL INFO Launch mode Group/CGMD
step 9 from 5115 at epoch 1
Loss: 1.07127046585083
step 19 from 5115 at epoch 1
Loss: 1.0069358348846436
step 29 from 5115 at epoch 1
Loss: 0.8561113476753235
step 39 from 5115 at epoch 1
Loss: 0.9998820424079895
step 49 from 5115 at epoch 1
Loss: 1.046589732170105
step 59 from 5115 at epoch 1